In [3]:
import asyncio
import httpx
import random
import string
import json
import time
import os
import argparse
import logging
from datetime import datetime
from collections import Counter

# --- Configuration & Setup ---

# Set up basic logging to provide clean, timestamped output.
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# The base URL for the Cloudflare API endpoint.
# The three random digits at the end will be generated for each request.
BASE_API_URL = "https://api.cloudflareclient.com/v0a{}/reg"

# --- Helper Functions ---

def gen_random_string(length: int) -> str:
    """Generates a random string of a given length from letters and digits."""
    return ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(length))

def get_request_headers() -> dict:
    """Returns a dictionary of headers for the API request."""
    return {
        "Content-Type": "application/json; charset=UTF-8",
        "Host": "api.cloudflareclient.com",
        "Connection": "Keep-Alive",
        "Accept-Encoding": "gzip",
        "User-Agent": "okhttp/3.12.1",
    }

def build_request_body(referrer_id: str) -> dict:
    """
    Builds the JSON body for the registration request.

    Args:
        referrer_id: The WARP+ client ID to use as the referrer.

    Returns:
        A dictionary representing the request body.
    """
    install_id = gen_random_string(22)
    return {
        "key": f"{gen_random_string(43)}=",
        "install_id": install_id,
        "fcm_token": f"{install_id}:APA91b{gen_random_string(134)}",
        "referrer": referrer_id,
        "warp_enabled": False,
        "tos": f"{datetime.now().isoformat()[:-3]}+02:00",
        "type": "Android",
        "locale": "es_ES",
    }

# --- Core Logic ---

async def run_worker(session: httpx.AsyncClient, referrer_id: str, results_counter: Counter):
    """
    A single worker task that sends one registration request.

    Args:
        session: An httpx.AsyncClient session to use for the request.
        referrer_id: The WARP+ client ID.
        results_counter: A Counter object to track successes and failures.
    """
    try:
        # Generate a random 3-digit number for the API endpoint version
        api_version = ''.join(random.choice(string.digits) for _ in range(3))
        url = BASE_API_URL.format(api_version)

        headers = get_request_headers()
        body = build_request_body(referrer_id)

        response = await session.post(url, json=body, headers=headers, timeout=10)

        if response.status_code == 200:
            logging.info("SUCCESS: Referrer profile updated. +1GB of data added.")
            results_counter.update(["success"])
        else:
            logging.error(f"FAILURE: Received status code {response.status_code}. Response: {response.text}")
            results_counter.update(["failure"])

    except httpx.RequestError as e:
        logging.error(f"HTTP Request Error: An error occurred while requesting {e.request.url!r}.")
        results_counter.update(["failure"])
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
        results_counter.update(["failure"])

# --- Main Execution ---

async def main(referrer_id: str, total_requests: int, delay_seconds: float):
    """
    Orchestrates the process of sending concurrent requests.

    Args:
        referrer_id: The WARP+ client ID.
        total_requests: The total number of requests to send.
        delay_seconds: The delay to wait between each request.
    """
    results = Counter()

    # Using an async context manager ensures the client is properly closed.
    async with httpx.AsyncClient() as session:
        tasks = []
        for i in range(total_requests):
            logging.info(f"Scheduling request {i + 1} of {total_requests}.")
            task = asyncio.create_task(run_worker(session, referrer_id, results))
            tasks.append(task)

            # Wait for the specified delay before scheduling the next request.
            await asyncio.sleep(delay_seconds)

        # Wait for all scheduled tasks to complete.
        await asyncio.gather(*tasks)

    logging.info("--- All requests completed ---")
    logging.info(f"Total successes: {results['success']} GB")
    logging.info(f"Total failures: {results['failure']}")


if __name__ == "__main__":
    parser = argparse.ArgumentParser(
        description="Advanced script to add referral data to a WARP+ account.",
        formatter_class=argparse.RawTextHelpFormatter,
        epilog="""
Example usage:
  python advanced_warp_script.py YOUR_CLIENT_ID_HERE -n 20 -d 5
  This will send 20 requests with a 5-second delay between each.
"""
    )

    parser.add_argument(
        "referrer_id",
        help="Your WARP+ Client ID."
    )
    parser.add_argument(
        "-n", "--requests",
        type=int,
        default=10,
        help="The total number of requests to send. (Default: 10)"
    )
    parser.add_argument(
        "-d", "--delay",
        type=float,
        default=20.0,
        help="The delay in seconds between scheduling each request. (Default: 20.0)"
    )

    args = parser.parse_args()

    # Run the main asynchronous event loop.
    try:
        asyncio.run(main(args.referrer_id, args.requests, args.delay))
    except KeyboardInterrupt:
        logging.info("\nProcess interrupted by user. Exiting.")



usage: colab_kernel_launcher.py [-h] [-n REQUESTS] [-d DELAY] referrer_id
colab_kernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2